In [1]:
%%time
import pandas as pd

from sklearn.metrics import roc_auc_score, make_scorer, accuracy_score
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler


from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from xgboost import XGBClassifier
from sklearn.feature_selection import RFE


from time import strftime, strptime, mktime, time, ctime

from sklearn import model_selection

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB 
from sklearn.svm import LinearSVC

from xgboost import XGBClassifier
import xgboost as xgb

from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier,\
ExtraTreesClassifier, GradientBoostingClassifier, VotingClassifier, RandomForestClassifier

from mlxtend.classifier import StackingClassifier, StackingCVClassifier
from mlxtend.feature_selection import ColumnSelector

from sklearn.pipeline import make_pipeline
import numpy as np

import gc



CPU times: user 537 ms, sys: 2.15 s, total: 2.69 s
Wall time: 422 ms


/home/shj16110/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
%%time
df = pd.read_csv('./train.csv')
print df.shape
test = pd.read_csv('./test.csv')
print test.shape

y = df.pop('Approved')
train_id = df.pop('ID')
test_id = test.pop('ID')

def helper(x):
    try:
        tmp = strptime(x, '%d/%m/%y')
        tmp = mktime(tmp)
    except:
        tmp = 563000400
    return tmp

df['Lead_Creation_Date'] = df['Lead_Creation_Date'].apply(helper)
test['Lead_Creation_Date'] = test['Lead_Creation_Date'].apply(helper)

df['DOB'] = df['DOB'].apply(helper)
test['DOB'] = test['DOB'].apply(helper)

df = df.fillna(axis=1, value=-1)
test = test.fillna(axis=1, value=-1)

df = pd.get_dummies(df)
test = pd.get_dummies(test)
test = test.reindex(columns = df.columns, fill_value=0)

test_o = test.copy()

(69713, 22)
(30037, 21)
CPU times: user 13.1 s, sys: 7.04 s, total: 20.2 s
Wall time: 20.2 s


In [3]:
# %%time

# rid = []

# for i in df:
#     if df[i].value_counts().head(1).values[0]>df.shape[0]*0.999:
#         rid.append(i)
# print len(rid), df.shape[1]-len(rid)

### from 186 features

In [4]:
from keep import keep
record = []

In [5]:
dic = {}
for i in keep:
    dic[i]=df[i].value_counts().head(1).values[0]


In [6]:
pic = pd.Series(dic).sort_values(ascending=True)
rm = list(pic.index)

In [7]:
len(rm)

186

In [114]:
RANDOM_STATE = 2018
params = {
    'min_child_weight': 0.8,
#     'eta': 0.1,
    'colsample_bytree': 0.8,
    'max_depth': 12,
    'subsample': 0.8,
    'alpha': 0.3,
    'lambda':0.9,
    'gamma': 0.3,
    'silent': 0,
    'learning_rate':0.02,
    'verbose_eval': True,
    'seed': RANDOM_STATE,
    'objective':"binary:logistic",
#     'eval_metric':'error'
#     'num_class':1
    'n_jobs':24,
    'eval_metric':'auc'
    
}



In [108]:
df.shape

(69713, 37416)

In [115]:
def search(i):
    global record
    
    update_rid = set(df.columns) - set(rm[:i])
    print len(df.columns) - len(update_rid)


    train = df.drop(update_rid, axis=1)
    test = test_o.drop(update_rid, axis=1)

    s = StandardScaler()
    
    train = s.fit_transform(train)
    test = s.transform(test)

    print train.shape, test.shape    
    
    m = lambda *x: [i.shape for i in x]
    X_train, X_test, y_train, y_test = train_test_split(train, y, stratify=y, test_size=0.2, random_state=23333)
    print m(X_train, y_train, X_test, y_test)

    d_train = xgb.DMatrix(X_train, label=y_train)
    d_test = xgb.DMatrix(X_test, label=y_test)    
    
    watchlist = [(d_train, 'train'), (d_test, 'test')]

    
    
    start=time()

    clf = xgb.train(params, d_train, 10000, watchlist, early_stopping_rounds=200, 
                    verbose_eval=200,  maximize=True,
                   )



    print '!!! used %.2f s'%(time()-start)    
    record.append([i, clf.best_iteration, clf.best_score])
    print record[-1]

In [93]:
clf.best_iteration

272

In [8]:
# for i in range(50, 151):
#     search(i)
# len(record)

In [129]:
rs = pd.DataFrame(record, columns=['feature','round','score'])
# rs

In [120]:
rs.to_csv('search1.csv',index=False)

In [127]:
rs[rs['score']>.859].sort_values(by=['score'],ascending=False)

,feature,round,score
78,128,243,0.862080
5,55,293,0.861089
35,85,301,0.860633
56,106,297,0.860536
60,110,318,0.860304
79,129,331,0.860284
71,121,357,0.860187
38,88,302,0.860144
65,115,310,0.860101
7,57,269,0.859971


## from here tune

In [79]:
update_rid = set(df.columns) - set(rm[:121])
print len(df.columns) - len(update_rid)


train = df.drop(update_rid, axis=1)
test = test_o.drop(update_rid, axis=1)

s = StandardScaler()

train = s.fit_transform(train)
test = s.transform(test)

print train.shape, test.shape    

m = lambda *x: [i.shape for i in x]
X_train, X_test, y_train, y_test = train_test_split(train, y, stratify=y, test_size=0.2, random_state=23333)
print m(X_train, y_train, X_test, y_test)

d_train = xgb.DMatrix(X_train, label=y_train)
d_test = xgb.DMatrix(X_test, label=y_test)    

watchlist = [(d_train, 'train'), (d_test, 'test')]


121
(69713, 121) (30037, 121)
[(55770, 121), (55770,), (13943, 121), (13943,)]


### from here

In [80]:
# RANDOM_STATE = 2018
# params = {
#     'min_child_weight': 0.9,
# #     'eta': 0.1,
#     'colsample_bytree': 0.9,
#     'max_depth': 21,
#     'subsample': 0.7,
# #     'alpha': 0.3,
# #     'lambda':0.9,
# #     'gamma': 0.3,
#     'silent': 0,
#     'learning_rate':0.001,
#     'verbose_eval': True,
#     'seed': RANDOM_STATE,
#     'objective':"binary:logistic",
#     'n_jobs':24,
#     'eval_metric':'auc'   
# }

In [99]:
RANDOM_STATE = 2018
params = {
    'min_child_weight': 0.8,
#     'eta': 0.1,
    'colsample_bytree': 0.8,
    'max_depth': 12,
    'subsample': 0.8,
    'alpha': 0.3,
    'lambda':0.9,
    'gamma': 0.3,
    'silent': 0,
    'learning_rate':0.02,
    'verbose_eval': True,
    'seed': RANDOM_STATE,
    'objective':"binary:logistic",
#     'eval_metric':'error'
#     'num_class':1
    'n_jobs':24,
    'eval_metric':'auc'
    
}



In [82]:
start=time()

clf = xgb.train(params, d_train, 357, watchlist, early_stopping_rounds=500, 
                verbose_eval=50,  maximize=True,
               )
                


'!!! used %.2f s'%(time()-start)

[0]	train-auc:0.74433	test-auc:0.752664
Multiple eval metrics have been passed: 'test-auc' will be used for early stopping.

Will train until test-auc hasn't improved in 500 rounds.
[50]	train-auc:0.872783	test-auc:0.837361
[100]	train-auc:0.918073	test-auc:0.845522
[150]	train-auc:0.953087	test-auc:0.851378
[200]	train-auc:0.973094	test-auc:0.855812
[250]	train-auc:0.98465	test-auc:0.858781
[300]	train-auc:0.990822	test-auc:0.859448
[350]	train-auc:0.994381	test-auc:0.860006


'!!! used 11.11 s'

In [83]:
clf.best_iteration, clf.best_score

(354, 0.86016)

In [109]:
update_rid = set(df.columns) - set(rm[:121])
print len(df.columns) - len(update_rid)


train = df.drop(update_rid, axis=1)
test = test_o.drop(update_rid, axis=1)

s = StandardScaler()

train = s.fit_transform(train)
test = s.transform(test)

print train.shape, test.shape    

# m = lambda *x: [i.shape for i in x]
# X_train, X_test, y_train, y_test = train_test_split(train, y, stratify=y, test_size=0.2, random_state=23333)
# print m(X_train, y_train, X_test, y_test)

d_train = xgb.DMatrix(train, label=y)

d_test = xgb.DMatrix(test)    

watchlist = [(d_train, 'train')]


121
(69713, 121) (30037, 121)


In [101]:
357//0.8

446.0

In [111]:
start=time()

clf = xgb.train(params, d_train, 446, watchlist, early_stopping_rounds=500, verbose_eval=50,  maximize=True)
                


'!!! used %.2f s'%(time()-start)

[0]	train-auc:0.755649
Will train until train-auc hasn't improved in 500 rounds.
[50]	train-auc:0.870563
[100]	train-auc:0.917515
[150]	train-auc:0.951629
[200]	train-auc:0.970433
[250]	train-auc:0.981741
[300]	train-auc:0.988032
[350]	train-auc:0.991788
[400]	train-auc:0.994404


'!!! used 16.99 s'

(324, 0.861728)

In [60]:
ind_rm = pd.Series(clf.get_fscore())[pd.Series(clf.get_fscore())<10].index
ind_rm

Index([u'f102', u'f112', u'f133', u'f138', u'f145', u'f146', u'f147', u'f151',
       u'f154', u'f164', u'f171', u'f172', u'f173', u'f176', u'f181', u'f32',
       u'f39', u'f45', u'f49', u'f50', u'f54', u'f58', u'f61', u'f65', u'f84',
       u'f86', u'f89', u'f91', u'f92'],
      dtype='object')

In [72]:
ind_rmint = map(lambda x: int(x[1:]), ind_rm)

In [73]:
update_rid = set(df.columns) - set(rm[:185]) 
print len(df.columns) - len(update_rid)


train = df.drop(update_rid, axis=1)
test = test_o.drop(update_rid, axis=1)



s = StandardScaler()

train = s.fit_transform(train)
test = s.transform(test)

185


In [75]:
train[:,ind_rmint]

array([[-0.0375199 , -0.24730837, -0.08456547, ..., -0.04338975,
        -0.04372035, -0.04255212],
       [-0.0375199 , -0.24730837, -0.08456547, ..., -0.04338975,
        -0.04372035, -0.04255212],
       [-0.0375199 , -0.24730837, -0.08456547, ..., -0.04338975,
        -0.04372035, -0.04255212],
       ...,
       [-0.0375199 , -0.24730837, -0.08456547, ..., -0.04338975,
        -0.04372035, -0.04255212],
       [-0.0375199 , -0.24730837, -0.08456547, ..., -0.04338975,
        -0.04372035, -0.04255212],
       [-0.0375199 , -0.24730837, -0.08456547, ..., -0.04338975,
        -0.04372035, -0.04255212]])

In [76]:
train = train[:,ind_rmint]
test = test[:,ind_rmint]



print train.shape, test.shape    

m = lambda *x: [i.shape for i in x]
X_train, X_test, y_train, y_test = train_test_split(train, y, stratify=y, test_size=0.2, random_state=23333)
print m(X_train, y_train, X_test, y_test)

d_train = xgb.DMatrix(X_train, label=y_train)
d_test = xgb.DMatrix(X_test, label=y_test)    

watchlist = [(d_train, 'train'), (d_test, 'test')]


(69713, 29) (30037, 29)
[(55770, 29), (55770,), (13943, 29), (13943,)]


In [112]:
%%time
dtest = xgb.DMatrix(test)
predb = clf.predict(dtest)

submit = pd.concat([pd.DataFrame(test_id.values, columns=['ID']), pd.DataFrame(predb, columns=['Approved'])], axis=1)
submit.shape

CPU times: user 1.84 s, sys: 22 ms, total: 1.87 s
Wall time: 134 ms


___

In [113]:
submit.to_csv('./submit/xgb_121_full.csv',index=False)

#### lb .852388

In [60]:
# clf.save_model('xgb_0.852586.model')

In [441]:
roc_auc_score?

In [442]:
submit2 = pd.concat([pd.DataFrame(test_id.values, columns=['ID']), pd.DataFrame(pred, columns=['Approved'])], axis=1)
submit2.shape

(30037, 2)

In [449]:
accuracy_score(y_test, [0]*y_test.shape[0])

0.9834325467976762

___

In [276]:
cross_val_score(RandomForestClassifier(n_estimators=50, n_jobs=24), df.iloc[:,:-1], df.iloc[:,-1], cv=5)

array([0.87807502, 1.        , 0.99992828, 1.        , 0.98759145])

In [76]:
fr = RandomForestClassifier(n_estimators=50)
fr

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [77]:
fr.fit(df.iloc[:,:-1], df.iloc[:,-1])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [ ]:
fr.predict_log_proba